In [1]:
import pandas as pd
import numpy as np
# import tensorflow as tf
from packages import wrappers, models, ta
from sklearn.linear_model import LinearRegression

In [2]:
btcusd_df = wrappers.get_df_from_cryptowatch(pair='ethusd')

In [3]:
end_date = '2021-08-15'
num_day = 365

In [4]:
btcusd_df = models.add_TA(btcusd_df)
prepped_df = models.prepare_for_model(btcusd_df, end_date, num_day)
prepped_df

,Target,Close,Volume,ATR_14,OBV
Date,,,,,
2020-08-14,433.67,432.39,2.006905e+07,25.852238,2.023126e+06
2020-08-15,431.35,433.67,2.717136e+07,25.792078,2.087150e+06
2020-08-16,422.62,431.35,3.055223e+07,25.873358,2.016556e+06
2020-08-17,407.57,422.62,3.295368e+07,25.603118,1.938928e+06
2020-08-18,416.20,407.57,3.725911e+07,26.143610,1.847700e+06
...,...,...,...,...,...
2021-08-10,3048.80,3164.50,1.602499e+08,195.612514,5.222545e+06
2021-08-11,3324.02,3048.80,1.848556e+08,199.969477,5.162558e+06
2021-08-12,3266.38,3324.02,1.681553e+08,206.524514,5.214775e+06


In [8]:
Xtrain, Xtest, ytrain, ytest = models.get_train_test(prepped_df)
display(Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape)

(365, 4)

(1, 4)

(365, 1)

(1, 1)

In [9]:
lr = LinearRegression().fit(Xtrain, Xtest)
prediction = lr.predict(ytrain)

ValueError: Found input variables with inconsistent numbers of samples: [365, 1]

In [ ]:
prediction

In [ ]:
# models.generate_model('btcusd', prepped_df, '2020-01-01', 200, force_overwrite=True)

In [ ]:
# pd.read_pickle('./models/btcusd_2020-01-01-linear-200/btcusd.pkl')

In [ ]:
# lr = models.load_model('btcusd', '2020-01-01', 200)